In [109]:
%load_ext autoreload
%autoreload 2
import warnings
import os
from etils import ecolab
import numpy as np
from asammdf import MDF
from pathlib import Path
import logging
from collections import  defaultdict
import pandas as pd

import  utils.can_log_utils as utls
from utils.dbc_db import DbcDb


logging.getLogger('asammdf').setLevel(logging.DEBUG)
%connect_info

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{"key":"95704ca3-bfc0-4966-b113-d5bacb6d5b80","signature_scheme":"hmac-sha256","transport":"tcp","ip":"127.0.0.1","hb_port":9005,"control_port":9006,"shell_port":9007,"stdin_port":9008,"iopub_port":9009,"kernel_name":"python3127jvsc74a57bd0e7c25480a53691149789381bdda99fb7f8d344b19173f1535ebe8bfb62cf4ee5"}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing c:\Users\meleshin\AppData\Roaming\jupyter\runtime\kernel-v3382a0206e312e6148cd3b2fc385f5eef904ffbf7.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [110]:
# set input data
dbc_dir=r'C:\d\data\OneDrive - Arrival\gdrive\work\VAN\DBC\ucs_dbc-0.12.0.0'
log_file1=r'input\r11_D19\R11_canlog_50640_20240926_142715.mf4'
log_file2=r'input\r12_D19\canlog_51938_20241107_122616.mf4'
file_export1=r'input\r11_D19\R11_canlog_50640_20240926_142715.xlsx'
file_export2=r'input\r12_D19\canlog_51938_20241107_122616.xlsx'
file_export_diff=r'output\mdf_diff_report.xlsx'
CAN_MAPS_FILE=r'input\VAN_CAN_MAPS.xlsx'
CAN_MAPS_SHEET_ECU='ECU_NAMES'
f_time1=[12,22] # selection of the log
f_time2=[12,22] # selection of the log

In [111]:
# prepare report file
utls.prepare_report_file(file_export1,dbc_dir,log_file1,f_time1)
utls.prepare_report_file(file_export2,dbc_dir,log_file2,f_time2)

In [112]:
utls.prepare_report_file(file_export_diff,dbc_dir,log_file1,f_time1,log_file2=log_file2,f_time2=f_time2)


In [113]:
# load databases
dbc_db=DbcDb(dbc_dir,False)
# (dbc_name_db,dbc_id_db,dbc_pgn_db)=utls.get_dbc_db(dbc_dir)
dbc_name_db_df=dbc_db.df
# load ECU names
ECU_NAME_MAP=pd.read_excel(CAN_MAPS_FILE,sheet_name=CAN_MAPS_SHEET_ECU,index_col='node_id_dec')
ECU_NAME_MAP.head();
ECU_NAME_MAP.loc[90,'node_name'];

total number of id: 1715
total number of dbc.msg: 3143
number of dublicated id: 1245
number of dublicated pgn: 1210


,node_name,node_id_hex
node_id_dec,,
3,IDU,3
17,ADAS,11
23,HMI,17
25,FBIO3,19
59,AVAS,3B


'DCU'

In [114]:
mdf1 = MDF(log_file1)
mdf2 = MDF(log_file2)
mdf1=mdf1.cut(f_time1[0],f_time1[1],whence=1,time_from_zero=1)
mdf2=mdf2.cut(f_time2[0],f_time2[1],whence=1,time_from_zero=1)

In [ ]:
mdf1_s = utls.mdf_get_can_msgs(mdf1,get_timestat=True)
mdf2_s = utls.mdf_get_can_msgs(mdf2,get_timestat=True)

In [203]:
# process prepare tables

(mdf1_s,mdf_df1)=utls.proc_mdf_all(mdf1,mdf1_s,dbc_db,ecu_name_map=ECU_NAME_MAP)
(mdf2_s,mdf_df2)=utls.proc_mdf_all(mdf2,mdf2_s,dbc_db,ecu_name_map=ECU_NAME_MAP)



print('log file1')
mdf_df1.attrs;
print('log file2')
mdf_df2.attrs;

print('log file1')
utls.print_stat_log_msgs(mdf_df1,dbc_db,file_export1)
print('log file2')
utls.print_stat_log_msgs(mdf_df2,dbc_db,file_export2)

print('log file1')
mdf_df1.head();
print('log file2')
mdf_df2.head();

{'mean': 10, 'cv': 10}
{'mean': 10, 'cv': 10}
log file1


{'mdf.file_history': '[FHBLOCK(time=2024-11-19 13:35:23.136528+04:00, comment=<FHcomment>\n    <TX>Cut from 12.000820711s to 22.000820711s</TX>\n    <tool_id>asammdf</tool_id>\n    <tool_vendor>asammdf</tool_vendor>\n    <tool_version>8.0.1</tool_version>\n</FHcomment>)]',
 'mdf.start_time': datetime.datetime(2024, 9, 26, 18, 27, 27, 751103),
 'mdf.name': WindowsPath('__new__.mf4')}

log file2


{'mdf.file_history': '[FHBLOCK(time=2024-11-19 13:35:23.229141+04:00, comment=<FHcomment>\n    <TX>Cut from 12.000004667s to 22.000004667s</TX>\n    <tool_id>asammdf</tool_id>\n    <tool_vendor>asammdf</tool_vendor>\n    <tool_version>8.0.1</tool_version>\n</FHcomment>)]',
 'mdf.start_time': datetime.datetime(2024, 11, 7, 16, 26, 28, 148050),
 'mdf.name': WindowsPath('__new__.mf4')}

log file1
log msgs total number: 650 
log number of id wo dbc: 177
log number dublicated id: 340

dbc_db total number unique dbc 138
log number of used dbc: 96

dbc_db total number of messages in db: 3143
log number of used messages: 597

SA with ECU:
{'FBIO4', 'HMI', 'ARVLC', 'BMS', 'CBIO2', None, 'CBIO3', 'STPCB', 'RRIO2', 'FBIO3', 'STPRD', 'DCU', 'STPLD', 'IDU', 'RRIO1', 'FBIO2', 'COMM', 'FBIO1', 'CBIO1', 'ADAS', 'INV'}

SA without ECU
{0.0, 1.0, 39.0, 51.0, 52.0, 53.0, 54.0, 56.0, 58.0, 84.0, 97.0, 98.0, 99.0, 101.0, 102.0, 103.0, 105.0, 106.0, 107.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 117.0, 118.0, 119.0, 254.0}


messages with dt stat violation
number of messages with dT mean violation: 48
  (2, 301359777, 1): 900.1490221111114%: BodyCan1.dbc::Swoboda_DBG2; PYLDSYSDBG.dbc::Swoboda_DBG2; UpperZonalCan.dbc::RideHeightSensorRaw
  (2, 368458915, 1): 900.1364657777777%: BodyCan1.dbc::ClimateTempData; Climate_System.dbc::ClimateTempData; UpperZonalCan.dbc::ClimateTempData
  (2, 

msg_pgn msg_pdu  msg_sa  msg_da   dt_mean  \
CAN_Bus msg_id    is_extended                                               
0       284222538 1             61668.0    PDU2    74.0     NaN  1.000020   
1       168820993 1            135168.0    PDU1     1.0     1.0  0.100001   
        217068963 1             61493.0    PDU2   163.0     NaN  1.000001   
2       32908311  1            128548.0    PDU2    23.0     NaN  1.000030   
        32908966  1            128550.0    PDU2   166.0     NaN  1.000023   

                                 dt_min    dt_max    dt_std     dt_cv  \
CAN_Bus msg_id    is_extended                                           
0       284222538 1            0.999860  1.000177  0.000100  0.009996   
1       168820993 1            0.099039  0.101023  0.000228  0.227534   
        217068963 1            0.999796  1.000243  0.000161  0.016138   
2       32908311  1            0.999882  1.000176  0.000091  0.009149   
        32908966  1            0.999846  1.000239  0.000144  0.014402   

                               dt_nobs  \
CAN_Bus msg_id    is_extended            
0       284222538 1                9.0   
1       168820993 1               99.0   
        217068963 1                9.0   
2       32908311  1                9.0   
        32908966  1                9.0   

                                                                             dbc  \
CAN_Bus msg_id    is_extended                                                      
0       284222538 1                                                           []   
1       168820993 1                                                           []   
        217068963 1            [{'dbc': 'ArrivalSecurity.dbc', 'msg': 'ImmoSt...   
2       32908311  1            [{'dbc': 'RollerShutter.dbc', 'msg': 'RSHMIR',...   
        32908966  1            [{'dbc': 'BodyCan1.dbc', 'msg': 'RSDS', 'is_j1...   

                              ecu_name_sa ecu_name_da  dt_stat_mean_diff  \
CAN_Bus msg_id    is_extended                                              
0       284222538 1                  COMM        None                NaN   
1       168820993 1                  None        None                NaN   
        217068963 1                 CBIO1        None         900.001199   
2       32908311  1                   HMI        None           0.003009   
        32908966  1                 RRIO1        None           0.002257   

                              dt_stat_mean_viol dt_stat_cv_viol  
CAN_Bus msg_id    is_extended                                    
0       284222538 1                        None           False  
1       168820993 1                        None           False  
        217068963 1                        True           False  
2       32908311  1                        None           False  
        32908966  1                        None           False

log file2


msg_pgn msg_pdu  msg_sa  msg_da   dt_mean  \
CAN_Bus msg_id    is_extended                                               
0       284222538 1             61668.0    PDU2    74.0     NaN  1.000035   
1       168820993 1            135168.0    PDU1     1.0     1.0  0.100012   
        217068963 1             61493.0    PDU2   163.0     NaN  1.000009   
2       32908311  1            128548.0    PDU2    23.0     NaN  1.000020   
        32908966  1            128550.0    PDU2   166.0     NaN  1.000025   

                                 dt_min    dt_max    dt_std     dt_cv  \
CAN_Bus msg_id    is_extended                                           
0       284222538 1            0.999879  1.000218  0.000113  0.011292   
1       168820993 1            0.097712  0.102129  0.000352  0.351583   
        217068963 1            0.999832  1.000204  0.000130  0.013009   
2       32908311  1            0.999909  1.000221  0.000086  0.008625   
        32908966  1            0.999913  1.000169  0.000077  0.007694   

                               dt_nobs  \
CAN_Bus msg_id    is_extended            
0       284222538 1                9.0   
1       168820993 1               99.0   
        217068963 1                9.0   
2       32908311  1                9.0   
        32908966  1                9.0   

                                                                             dbc  \
CAN_Bus msg_id    is_extended                                                      
0       284222538 1                                                           []   
1       168820993 1                                                           []   
        217068963 1            [{'dbc': 'ArrivalSecurity.dbc', 'msg': 'ImmoSt...   
2       32908311  1            [{'dbc': 'RollerShutter.dbc', 'msg': 'RSHMIR',...   
        32908966  1            [{'dbc': 'BodyCan1.dbc', 'msg': 'RSDS', 'is_j1...   

                              ecu_name_sa ecu_name_da  dt_stat_mean_diff  \
CAN_Bus msg_id    is_extended                                              
0       284222538 1                  COMM        None                NaN   
1       168820993 1                  None        None                NaN   
        217068963 1                 CBIO1        None         900.009226   
2       32908311  1                   HMI        None           0.001997   
        32908966  1                 RRIO1        None           0.002549   

                              dt_stat_mean_viol dt_stat_cv_viol  
CAN_Bus msg_id    is_extended                                    
0       284222538 1                        None           False  
1       168820993 1                        None           False  
        217068963 1                        True           False  
2       32908311  1                        None           False  
        32908966  1                        None           False

In [156]:
# prepare can matrix
utls.get_can_com_matrix(mdf_df1,file_export=file_export1)
utls.get_can_com_matrix(mdf_df2,file_export=file_export2)

CAN MATRIX reports written: input\r11_D19\R11_canlog_50640_20240926_142715.xlsx
CAN MATRIX reports written: input\r12_D19\canlog_51938_20241107_122616.xlsx


(                        CAN0  CAN1  CAN2  CAN3  CAN4  CAN5  CAN6  CAN7  CAN8
 ACU_Arrival.dbc          NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0   NaN
 ADAS_Data.dbc            NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN
 ADAS_J1939.dbc           NaN   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0
 ADAS_Telemetry.dbc       NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0   NaN
 ALARMDRV.dbc             NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN
 ...                      ...   ...   ...   ...   ...   ...   ...   ...   ...
 j1939_HVAC.dbc           NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN
 j1939_TPMS.dbc           NaN   NaN   1.0   NaN   NaN   NaN   NaN   1.0   NaN
 j1939_system.dbc         NaN   1.0   1.0   NaN   NaN   NaN   NaN   1.0   1.0
 maintenance_status.dbc   NaN   1.0   1.0   NaN   NaN   NaN   NaN   1.0   NaN
 module_info.dbc          1.0   1.0   1.0   NaN   NaN   NaN   NaN   1.0   1.0
 
 [96 rows x 9 columns],
                                      

In [119]:
mdf_df_c=utls.mdf_diff_report(mdf_df1,mdf_df2,file_export_diff)
mdf_df_c.head();

report written into output\mdf_diff_report.xlsx


CAN_Bus     msg_id  is_extended   msg_pgn  \
CAN_Bus msg_id    is_extended                                              
2       418119589 1                  2  418119589            1   60160.0   
7       193       0                  7        193            0       NaN   
2       418119514 1                  2  418119514            1   60160.0   
        33048485  1                  2   33048485            1  129095.0   
7       418119441 1                  7  418119441            1   60160.0   

                               msg_sa  f1.dt_mean    f1.dt_cv  f1.dt_nobs  \
CAN_Bus msg_id    is_extended                                               
2       418119589 1             165.0    0.235035  139.602946        39.0   
7       193       0               NaN    0.010001    7.226660       999.0   
2       418119514 1              90.0    0.091248  124.944707       109.0   
        33048485  1             165.0    0.260553  106.987896        38.0   
7       418119441 1              17.0    0.118813  149.804006        79.0   

                               f2.dt_mean    f2.dt_cv  f2.dt_nobs _merge  \
CAN_Bus msg_id    is_extended                                              
2       418119589 1              0.476643   93.003305        19.0   both   
7       193       0              0.004998   93.950202      1999.0   both   
2       418119514 1              0.106079  143.674602        89.0   both   
        33048485  1              0.293755   91.248314        32.0   both   
7       418119441 1              0.106079  143.583902        89.0   both   

                                                           STATUS  \
CAN_Bus msg_id    is_extended                                       
2       418119589 1            +DT_MEAN DIFF+DT_CV DIFF+NOBS DIFF   
7       193       0            +DT_MEAN DIFF+DT_CV DIFF+NOBS DIFF   
2       418119514 1            +DT_MEAN DIFF+DT_CV DIFF+NOBS DIFF   
        33048485  1            +DT_MEAN DIFF+DT_CV DIFF+NOBS DIFF   
7       418119441 1                       +DT_MEAN DIFF+NOBS DIFF   

                               diff.dt_mean  diff.dt_cv  diff.dt_nobs  \
CAN_Bus msg_id    is_extended                                           
2       418119589 1                  102.80       33.38         51.28   
7       193       0                   50.02     1200.05        100.10   
2       418119514 1                   16.25       14.99         18.35   
        33048485  1                   12.74       14.71         15.79   
7       418119441 1                   10.72        4.15         12.66   

                                                                            dbc1  \
CAN_Bus msg_id    is_extended                                                      
2       418119589 1            [{'dbc': 'ADAS_J1939.dbc', 'msg': 'TPDT', 'is_...   
7       193       0            [{'dbc': 'NTO_ARRIVAL_MANDO_CAN_SPEC_P.dbc', '...   
2       418119514 1            [{'dbc': 'ADAS_J1939.dbc', 'msg': 'TPDT', 'is_...   
        33048485  1            [{'dbc': 'BodyCan1.dbc', 'msg': 'SLS', 'is_j19...   
7       418119441 1            [{'dbc': 'ADAS_J1939.dbc', 'msg': 'TPDT', 'is_...   

                                                                            dbc2  
CAN_Bus msg_id    is_extended                                                     
2       418119589 1            [{'dbc': 'ADAS_J1939.dbc', 'msg': 'TPDT', 'is_...  
7       193       0            [{'dbc': 'NTO_ARRIVAL_MANDO_CAN_SPEC_P.dbc', '...  
2       418119514 1            [{'dbc': 'ADAS_J1939.dbc', 'msg': 'TPDT', 'is_...  
        33048485  1            [{'dbc': 'BodyCan1.dbc', 'msg': 'SLS', 'is_j19...  
7       418119441 1            [{'dbc': 'ADAS_J1939.dbc', 'msg': 'TPDT', 'is_...